In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install  -U bitsandbytes


In [ ]:
!pip install  -U git+https://github.com/huggingface/transformers.git 

In [ ]:
!pip install  -U git+https://github.com/huggingface/peft.git
!pip install  -U git+https://github.com/huggingface/accelerate.git
!pip install  datasets

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-3b")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-3b",quantization_config=nf4_config,)

In [ ]:
! pip install peft

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=4, 
    lora_alpha=16, 
            target_modules=["query_key_value"],

    lora_dropout=0.05, 
    bias="none", 
    
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


In [ ]:
data=pd.read_csv("/kaggle/input/arabic-samsum-dialogsum/arabic_dialog.csv")

In [ ]:
from torch.utils.data import DataLoader, Dataset


In [ ]:
"""class SummarizationDataset(Dataset):
    def __init__(self, tokenized_dataset):
        a = tokenizer( (data['dialogue']+data['summary']).tolist(), truncation=True, padding='max_length', return_tensors="pt", max_length=512)
        self.dataset=  [
            {
            
                            "input_ids":inputs,
                            "attention_mask": att,
                            "labels":inputs
                        } 
                            for inputs,att in zip(a['input_ids'],a['attention_mask'])
        ]
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]"""

In [ ]:
class SummarizationDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.dialogue = tokenized_dataset['dialogue']
        self.summary = tokenized_dataset['summary']
        self.tokenized_dataset=tokenized_dataset
    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, idx):
        a=tokenizer(self.dialogue[idx] +f"\n\nSummarize the previous text in three sentences:\n\n"+self.summary[idx], truncation=True, padding='max_length', return_tensors="pt", max_length=512)
        return {
            
                        "input_ids": a["input_ids"][0],
            "attention_mask": a["attention_mask"][0],
            "labels":a['input_ids'][0]
        }

In [ ]:
dataset_train=SummarizationDataset(data)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
from torch.utils.data import  random_split

In [ ]:
dataset_train,dataset_valid,dataset_test=random_split(dataset_train,[.85,.075,.075])

In [ ]:
import torch

In [ ]:
model.enable_input_require_grads()

In [ ]:
import wandb
wandb.login(key="14459c516497ab76a78f7fc1278bfe60d301d250")

In [ ]:
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=26,
    save_steps=200,
    learning_rate=2e-5,
            fp16=True
,
    weight_decay=0.001,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)
trainer = Trainer(
    model=model,
    args=train_params,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid
)

In [ ]:
trainer.train()

In [ ]:
torch.save(model,'bloomz.pt')